# Module 4 - Task
PySpark Dataframe with Pandas usage with yet another annoying Covid Dataset

In [0]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import time

import pyspark # only run this after findspark.init()
from pyspark.sql import SparkSession, SQLContext
from pyspark.context import SparkContext
from pyspark.sql.functions import * 
from pyspark.sql.types import * 

In [0]:
# Initiate the Spark Session
spark = SparkSession.builder.appName('covid-annoying-sample').getOrCreate()

In [0]:
spark

SparkSession - in-memory 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.6 
 Master 
 local[*] 
 AppName 
 covid-example

## Data
Full circle COVID data sample.

### 1. Basic Functions

#### [1] Load (Read) the data

In [0]:
cases = spark.read.load("dbfs:/FileStore/Module4/Case.csv",
                        format="csv", 
                        sep=",", 
                        inferSchema="true", 
                        header="true")

In [0]:
# First few rows in the file
cases.show()

+--------+--------+---------------+-----+--------------------+---------+---------+----------+
 case_id|province| city|group| infection_case|confirmed| latitude| longitude|
+--------+--------+---------------+-----+--------------------+---------+---------+----------+
 1000001| Seoul| Yongsan-gu| true| Itaewon Clubs| 139|37.538621|126.992652|
 1000002| Seoul| Gwanak-gu| true| Richway| 119| 37.48208|126.901384|
 1000003| Seoul| Guro-gu| true| Guro-gu Call Center| 95|37.508163|126.884387|
 1000004| Seoul| Yangcheon-gu| true|Yangcheon Table T...| 43|37.546061|126.874209|
 1000005| Seoul| Dobong-gu| true| Day Care Center| 43|37.679422|127.044374|
 1000006| Seoul| Guro-gu| true|Manmin Central Ch...| 41|37.481059|126.894343|
 1000007| Seoul|from other city| true|SMR Newly Planted...| 36| -| -|
 1000008| Seoul| Dongdaemun-gu| true| Dongan Church| 17|37.592888|127.056766|
 1000009| Seoul|from other city| true|Coupang Logistics...| 25| -| -|
 1000010| Seoul| Gwanak-gu| true| Wangsung Church| 30|37.481735|126.930121|
 1000011| Seoul| Eunpyeong-gu| true|Eunpyeong St. Mar...| 14| 37.63369| 126.9165|
 1000012| Seoul| Seongdong-gu| true| Seongdong-gu APT| 13| 37.55713| 127.0403|
 1000013| Seoul| Jongno-gu| true|Jongno Community ...| 10| 37.57681| 127.006|
 1000014| Seoul| Gangnam-gu| true|Samsung Medical C...| 7| 37.48825| 127.08559|
 1000015| Seoul| Jung-gu| true|Jung-gu Fashion C...| 7|37.562405|126.984377|
 1000016| Seoul| Seodaemun-gu| true| Yeonana News Class| 5|37.558147|126.943799|
 1000017| Seoul| Jongno-gu| true|Korea Campus Crus...| 7|37.594782|126.968022|
 1000018| Seoul| Gangnam-gu| true|Gangnam Yeoksam-d...| 6| -| -|
 1000019| Seoul|from other city| true|Daejeon door-to-d...| 1| -| -|
 1000020| Seoul| Geumcheon-gu| true|Geumcheon-gu rice...| 6| -| -|
+--------+--------+---------------+-----+--------------------+---------+---------+----------+
only showing top 20 rows

It looks ok right now, but sometimes as we the number of columns increases, the formatting becomes not too great. I have noticed that the following trick helps in displaying in pandas format in my Jupyter Notebook. 

The **.toPandas()** function converts a **Spark Dataframe** into a **Pandas Dataframe**, which is much easier to play with.

In [0]:
cases.limit(10).toPandas()

Out[5]:

,case_id,province,city,group,infection_case,confirmed,latitude,longitude
0,1000001,Seoul,Yongsan-gu,True,Itaewon Clubs,139,37.538621,126.992652
1,1000002,Seoul,Gwanak-gu,True,Richway,119,37.48208,126.901384
2,1000003,Seoul,Guro-gu,True,Guro-gu Call Center,95,37.508163,126.884387
3,1000004,Seoul,Yangcheon-gu,True,Yangcheon Table Tennis Club,43,37.546061,126.874209
4,1000005,Seoul,Dobong-gu,True,Day Care Center,43,37.679422,127.044374
5,1000006,Seoul,Guro-gu,True,Manmin Central Church,41,37.481059,126.894343
6,1000007,Seoul,from other city,True,SMR Newly Planted Churches Group,36,-,-
7,1000008,Seoul,Dongdaemun-gu,True,Dongan Church,17,37.592888,127.056766
8,1000009,Seoul,from other city,True,Coupang Logistics Center,25,-,-
9,1000010,Seoul,Gwanak-gu,True,Wangsung Church,30,37.481735,126.930121


#### [2] Change Column Names

In [0]:
# single column
cases = cases.withColumnRenamed("infection_case","infection_source")

In [0]:
# change all columns name
cases = cases.toDF(*['case_id', 'province', 'city', 'group', 'infection_case', 'confirmed',
       'latitude', 'longitude'])

In [0]:
cases.show(4)

+-------+--------+------------+-----+--------------------+---------+---------+----------+
case_id|province| city|group| infection_case|confirmed| latitude| longitude|
+-------+--------+------------+-----+--------------------+---------+---------+----------+
1000001| Seoul| Yongsan-gu| true| Itaewon Clubs| 139|37.538621|126.992652|
1000002| Seoul| Gwanak-gu| true| Richway| 119| 37.48208|126.901384|
1000003| Seoul| Guro-gu| true| Guro-gu Call Center| 95|37.508163|126.884387|
1000004| Seoul|Yangcheon-gu| true|Yangcheon Table T...| 43|37.546061|126.874209|
+-------+--------+------------+-----+--------------------+---------+---------+----------+
only showing top 4 rows

#### [3] Change Column Names

In [0]:
# We can select a subset of columns using the **select** 
cases = cases.select('province','city','infection_case','confirmed')
cases.show()

+--------+---------------+--------------------+---------+
|province|           city|      infection_case|confirmed|
+--------+---------------+--------------------+---------+
|   Seoul|     Yongsan-gu|       Itaewon Clubs|      139|
|   Seoul|      Gwanak-gu|             Richway|      119|
|   Seoul|        Guro-gu| Guro-gu Call Center|       95|
|   Seoul|   Yangcheon-gu|Yangcheon Table T...|       43|
|   Seoul|      Dobong-gu|     Day Care Center|       43|
|   Seoul|        Guro-gu|Manmin Central Ch...|       41|
|   Seoul|from other city|SMR Newly Planted...|       36|
|   Seoul|  Dongdaemun-gu|       Dongan Church|       17|
|   Seoul|from other city|Coupang Logistics...|       25|
|   Seoul|      Gwanak-gu|     Wangsung Church|       30|
|   Seoul|   Eunpyeong-gu|Eunpyeong St. Mar...|       14|
|   Seoul|   Seongdong-gu|    Seongdong-gu APT|       13|
|   Seoul|      Jongno-gu|Jongno Community ...|       10|
|   Seoul|     Gangnam-gu|Samsung Medical C...|        7|
|   Seoul|    

#### [4] Sort by Column

In [0]:
# Simple sort
cases.sort("confirmed").show()

+-----------------+---------------+--------------------+---------+
|         province|           city|      infection_case|confirmed|
+-----------------+---------------+--------------------+---------+
|          Jeju-do|              -|contact with patient|        0|
|       Gangwon-do|              -|contact with patient|        0|
|            Seoul|     Gangseo-gu|SJ Investment Cal...|        0|
|            Busan|from other city|Cheongdo Daenam H...|        1|
|     Jeollabuk-do|from other city|  Shincheonji Church|        1|
|            Seoul|from other city|Anyang Gunpo Past...|        1|
|            Seoul|     Gangnam-gu|Gangnam Dongin Ch...|        1|
|           Sejong|from other city|  Shincheonji Church|        1|
|     Jeollanam-do|from other city|  Shincheonji Church|        1|
|          Jeju-do|from other city|       Itaewon Clubs|        1|
|            Seoul|from other city|Daejeon door-to-d...|        1|
|            Seoul|              -|         Orange Life|      

In [0]:
# Descending Sort
from pyspark.sql import functions as F

cases.sort(F.desc("confirmed")).show()

+-----------------+---------------+--------------------+---------+
|         province|           city|      infection_case|confirmed|
+-----------------+---------------+--------------------+---------+
|            Daegu|         Nam-gu|  Shincheonji Church|     4511|
|            Daegu|              -|contact with patient|      917|
|            Daegu|              -|                 etc|      747|
| Gyeongsangbuk-do|from other city|  Shincheonji Church|      566|
|      Gyeonggi-do|              -|     overseas inflow|      305|
|            Seoul|              -|     overseas inflow|      298|
|            Daegu|   Dalseong-gun|Second Mi-Ju Hosp...|      196|
| Gyeongsangbuk-do|              -|contact with patient|      190|
|            Seoul|              -|contact with patient|      162|
|            Seoul|     Yongsan-gu|       Itaewon Clubs|      139|
| Gyeongsangbuk-do|              -|                 etc|      133|
|            Daegu|         Seo-gu|Hansarang Convale...|      

#### [5] Change Column Type

In [0]:
from pyspark.sql.types import DoubleType, IntegerType, StringType

cases = cases.withColumn('confirmed', F.col('confirmed').cast(IntegerType()))
cases = cases.withColumn('city', F.col('city').cast(StringType()))

cases.show()

+--------+---------------+--------------------+---------+
|province|           city|      infection_case|confirmed|
+--------+---------------+--------------------+---------+
|   Seoul|     Yongsan-gu|       Itaewon Clubs|      139|
|   Seoul|      Gwanak-gu|             Richway|      119|
|   Seoul|        Guro-gu| Guro-gu Call Center|       95|
|   Seoul|   Yangcheon-gu|Yangcheon Table T...|       43|
|   Seoul|      Dobong-gu|     Day Care Center|       43|
|   Seoul|        Guro-gu|Manmin Central Ch...|       41|
|   Seoul|from other city|SMR Newly Planted...|       36|
|   Seoul|  Dongdaemun-gu|       Dongan Church|       17|
|   Seoul|from other city|Coupang Logistics...|       25|
|   Seoul|      Gwanak-gu|     Wangsung Church|       30|
|   Seoul|   Eunpyeong-gu|Eunpyeong St. Mar...|       14|
|   Seoul|   Seongdong-gu|    Seongdong-gu APT|       13|
|   Seoul|      Jongno-gu|Jongno Community ...|       10|
|   Seoul|     Gangnam-gu|Samsung Medical C...|        7|
|   Seoul|    

#### [6] Filter

In [0]:
# We can filter a data frame using multiple conditions using AND(&), OR(|) and NOT(~) conditions. 
# For example, we may want to find out all the different infection_case in Daegu with more than 10 confirmed cases.
# Lovely province :)
cases.filter((cases.confirmed>10) & (cases.province=='Daegu')).show()

+-------+--------+------------+-----+--------------------+---------+---------+----------+
case_id|province| city|group| infection_case|confirmed| latitude| longitude|
+-------+--------+------------+-----+--------------------+---------+---------+----------+
1200001| Daegu| Nam-gu| true| Shincheonji Church| 4511| 35.84008| 128.5667|
1200002| Daegu|Dalseong-gun| true|Second Mi-Ju Hosp...| 196|35.857375|128.466651|
1200003| Daegu| Seo-gu| true|Hansarang Convale...| 124|35.885592|128.556649|
1200004| Daegu|Dalseong-gun| true|Daesil Convalesce...| 101|35.857393|128.466653|
1200005| Daegu| Dong-gu| true| Fatima Hospital| 39| 35.88395|128.624059|
1200008| Daegu| -|false| overseas inflow| 41| -| -|
1200009| Daegu| -|false|contact with patient| 917| -| -|
1200010| Daegu| -|false| etc| 747| -| -|
+-------+--------+------------+-----+--------------------+---------+---------+----------+

#### [7] GroupBy

In [0]:
from pyspark.sql import functions as F

cases.groupBy(["province","city"]).agg(F.sum("confirmed") ,F.max("confirmed")).show()

+----------------+---------------+--------------+--------------+
 province| city|sum(confirmed)|max(confirmed)|
+----------------+---------------+--------------+--------------+
Gyeongsangnam-do| Jinju-si| 9| 9|
 Seoul| Guro-gu| 139| 95|
 Seoul| Gangnam-gu| 18| 7|
 Daejeon| -| 100| 55|
 Jeollabuk-do|from other city| 6| 3|
Gyeongsangnam-do|Changnyeong-gun| 7| 7|
 Seoul| -| 561| 298|
 Jeju-do|from other city| 1| 1|
Gyeongsangbuk-do| -| 345| 190|
Gyeongsangnam-do| Geochang-gun| 18| 10|
Gyeongsangbuk-do| Gumi-si| 10| 10|
 Incheon|from other city| 117| 53|
 Busan| -| 85| 36|
 Daegu| Seo-gu| 124| 124|
 Busan| Suyeong-gu| 5| 5|
 Gyeonggi-do| Uijeongbu-si| 50| 50|
 Seoul| Yongsan-gu| 139| 139|
 Daegu| -| 1705| 917|
 Seoul| Seodaemun-gu| 5| 5|
 Gyeonggi-do| Seongnam-si| 94| 67|
+----------------+---------------+--------------+--------------+
only showing top 20 rows

Or if we don’t like the new column names, we can use the **alias** keyword to rename columns in the agg command itself.

In [0]:
cases.groupBy(["province","city"]).agg(
    F.sum("confirmed").alias("TotalConfirmed"),\
    F.max("confirmed").alias("MaxFromOneConfirmedCase")\
    ).show()

+----------------+---------------+--------------+-----------------------+
|        province|           city|TotalConfirmed|MaxFromOneConfirmedCase|
+----------------+---------------+--------------+-----------------------+
|Gyeongsangnam-do|       Jinju-si|             9|                      9|
|           Seoul|        Guro-gu|           139|                     95|
|           Seoul|     Gangnam-gu|            18|                      7|
|         Daejeon|              -|           100|                     55|
|    Jeollabuk-do|from other city|             6|                      3|
|Gyeongsangnam-do|Changnyeong-gun|             7|                      7|
|           Seoul|              -|           561|                    298|
|         Jeju-do|from other city|             1|                      1|
|Gyeongsangbuk-do|              -|           345|                    190|
|Gyeongsangnam-do|   Geochang-gun|            18|                     10|
|Gyeongsangbuk-do|        Gumi-si|    

#### [8] Joins

In [0]:
# adding region file which contains region information such as elementary_school_count, elderly_population_ratio, etc.
regions = spark.read.load("dbfs:/FileStore/Module4/Region.csv",
                          format="csv", 
                          sep=",", 
                          inferSchema="true", 
                          header="true")

regions.limit(10).toPandas()

Out[15]:

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739
1,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088
2,10020,Seoul,Gangdong-gu,37.530492,127.123837,27,32,0,1.54,14.55,5.4,1023
3,10030,Seoul,Gangbuk-gu,37.639938,127.025508,14,21,0,0.67,19.49,8.5,628
4,10040,Seoul,Gangseo-gu,37.551166,126.849506,36,56,1,1.17,14.39,5.7,1080
5,10050,Seoul,Gwanak-gu,37.478290,126.951502,22,33,1,0.89,15.12,4.9,909
6,10060,Seoul,Gwangjin-gu,37.538712,127.082366,22,33,3,1.16,13.75,4.8,723
7,10070,Seoul,Guro-gu,37.495632,126.887650,26,34,3,1.00,16.21,5.7,741
8,10080,Seoul,Geumcheon-gu,37.456852,126.895229,18,19,0,0.96,16.15,6.7,475
9,10090,Seoul,Nowon-gu,37.654259,127.056294,42,66,6,1.39,15.40,7.4,952


In [0]:
# Left Join 'Case' with 'Region' on Province and City column
cases = cases.join(regions, ['province','city'],how='left')
cases.limit(10).toPandas()

Out[16]:

,province,city,case_id,group,infection_case,confirmed,latitude,longitude,code,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,Seoul,Yongsan-gu,1000001,True,Itaewon Clubs,139,37.538621,126.992652,10210.0,37.532768,126.990021,15.0,13.0,1.0,0.68,16.87,6.5,435.0
1,Seoul,Gwanak-gu,1000002,True,Richway,119,37.48208,126.901384,10050.0,37.478290,126.951502,22.0,33.0,1.0,0.89,15.12,4.9,909.0
2,Seoul,Guro-gu,1000003,True,Guro-gu Call Center,95,37.508163,126.884387,10070.0,37.495632,126.887650,26.0,34.0,3.0,1.00,16.21,5.7,741.0
3,Seoul,Yangcheon-gu,1000004,True,Yangcheon Table Tennis Club,43,37.546061,126.874209,10190.0,37.517189,126.866618,30.0,43.0,0.0,2.26,13.55,5.5,816.0
4,Seoul,Dobong-gu,1000005,True,Day Care Center,43,37.679422,127.044374,10100.0,37.668952,127.047082,23.0,26.0,1.0,0.95,17.89,7.2,485.0
5,Seoul,Guro-gu,1000006,True,Manmin Central Church,41,37.481059,126.894343,10070.0,37.495632,126.887650,26.0,34.0,3.0,1.00,16.21,5.7,741.0
6,Seoul,from other city,1000007,True,SMR Newly Planted Churches Group,36,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Seoul,Dongdaemun-gu,1000008,True,Dongan Church,17,37.592888,127.056766,10110.0,37.574552,127.039721,21.0,31.0,4.0,1.06,17.26,6.7,832.0
8,Seoul,from other city,1000009,True,Coupang Logistics Center,25,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Seoul,Gwanak-gu,1000010,True,Wangsung Church,30,37.481735,126.930121,10050.0,37.478290,126.951502,22.0,33.0,1.0,0.89,15.12,4.9,909.0


### 2. Use SQL with DataFrames

We first register the cases dataframe to a temporary table cases_table on which we can run SQL operations. As you can see, the result of the SQL select statement is again a Spark Dataframe.

All complex SQL queries like GROUP BY, HAVING, AND ORDER BY clauses can be applied in 'Sql' function

In [0]:
cases.registerTempTable('cases_table')
newDF = spark.sql('select * from cases_table where confirmed > 100')
newDF.show()

+-----------------+---------------+-------+-----+--------------------+---------+---------+----------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
 province| city|case_id|group| infection_case|confirmed| latitude| longitude| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|
+-----------------+---------------+-------+-----+--------------------+---------+---------+----------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
 Seoul| Yongsan-gu|1000001| true| Itaewon Clubs| 139|37.538621|126.992652|10210|37.532768|126.990021| 15| 13| 1| 0.68| 16.87| 6.5| 435|
 Seoul| Gwanak-gu|1000002| true| Richway| 119| 37.48208|126.901384|10050| 37.47829|126.951502| 22| 33| 1| 0.89| 15.12| 4.9| 909|
 Seoul| -|1000036|false| overseas inflow| 298| -| -| null| null| null| null| null| null| null| null| null| null|
 Seoul| -|1000037|false|contact with patient| 162| -| -| null| null| null| null| null| null| null| null| null| null|
 Daegu| Nam-gu|1200001| true| Shincheonji Church| 4511| 35.84008| 128.5667|12010| 35.8463|128.597723| 11| 15| 2| 0.85| 22.49| 10.4| 345|
 Daegu| Dalseong-gun|1200002| true|Second Mi-Ju Hosp...| 196|35.857375|128.466651|12030| 35.77475|128.431314| 32| 47| 1| 1.51| 12.11| 5.4| 361|
 Daegu| Seo-gu|1200003| true|Hansarang Convale...| 124|35.885592|128.556649|12060|35.871993|128.559182| 17| 23| 0| 0.83| 21.29| 10.1| 374|
 Daegu| Dalseong-gun|1200004| true|Daesil Convalesce...| 101|35.857393|128.466653|12030| 35.77475|128.431314| 32| 47| 1| 1.51| 12.11| 5.4| 361|
 Daegu| -|1200009|false|contact with patient| 917| -| -| null| null| null| null| null| null| null| null| null| null|
 Daegu| -|1200010|false| etc| 747| -| -| null| null| null| null| null| null| null| null| null| null|
 Gyeonggi-do| -|2000020|false| overseas inflow| 305| -| -| null| null| null| null| null| null| null| null| null| null|
Chungcheongnam-do| Cheonan-si|4100001| true|gym facility in C...| 103| 36.81503| 127.1139|41120| 36.81498|127.113868| 75| 112| 6| 1.91| 10.42| 4.5| 1069|
 Gyeongsangbuk-do|from other city|6000001| true| Shincheonji Church| 566| -| -| null| null| null| null| null| null| null| null| null| null|
 Gyeongsangbuk-do| Cheongdo-gun|6000002| true|Cheongdo Daenam H...| 119| 35.64887| 128.7368|60200|35.647361|128.734382| 11| 14| 0| 0.63| 36.55| 21.0| 85|
 Gyeongsangbuk-do| -|6000012|false|contact with patient| 190| -| -| null| null| null| null| null| null| null| null| null| null|
 Gyeongsangbuk-do| -|6000013|false| etc| 133| -| -| null| null| null| null| null| null| null| null| null| null|
+-----------------+---------------+-------+-----+--------------------+---------+---------+----------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+

### 3. Create New Columns

There are many ways that you can use to create a column in a PySpark Dataframe.

#### [1] Using Spark Native Functions

We can use .withcolumn along with PySpark SQL functions to create a new column. In essence, you can find String functions, Date functions, and Math functions already implemented using Spark functions. Our first function, the F.col function gives us access to the column. So if we wanted to add 100 to a column, we could use F.col as:

In [0]:
import pyspark.sql.functions as F

casesWithNewConfirmed = cases.withColumn("NewConfirmed", 100 + F.col("confirmed"))
casesWithNewConfirmed.show()

+--------+---------------+-------+-----+--------------------+---------+---------+----------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+------------+
province| city|case_id|group| infection_case|confirmed| latitude| longitude| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|NewConfirmed|
+--------+---------------+-------+-----+--------------------+---------+---------+----------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+------------+
 Seoul| Yongsan-gu|1000001| true| Itaewon Clubs| 139|37.538621|126.992652|10210|37.532768|126.990021| 15| 13| 1| 0.68| 16.87| 6.5| 435| 239|
 Seoul| Gwanak-gu|1000002| true| Richway| 119| 37.48208|126.901384|10050| 37.47829|126.951502| 22| 33| 1| 0.89| 15.12| 4.9| 909| 219|
 Seoul| Guro-gu|1000003| true| Guro-gu Call Center| 95|37.508163|126.884387|10070|37.495632| 126.88765| 26| 34| 3| 1.0| 16.21| 5.7| 741| 195|
 Seoul| Yangcheon-gu|1000004| true|Yangcheon Table T...| 43|37.546061|126.874209|10190|37.517189|126.866618| 30| 43| 0| 2.26| 13.55| 5.5| 816| 143|
 Seoul| Dobong-gu|1000005| true| Day Care Center| 43|37.679422|127.044374|10100|37.668952|127.047082| 23| 26| 1| 0.95| 17.89| 7.2| 485| 143|
 Seoul| Guro-gu|1000006| true|Manmin Central Ch...| 41|37.481059|126.894343|10070|37.495632| 126.88765| 26| 34| 3| 1.0| 16.21| 5.7| 741| 141|
 Seoul|from other city|1000007| true|SMR Newly Planted...| 36| -| -| null| null| null| null| null| null| null| null| null| null| 136|
 Seoul| Dongdaemun-gu|1000008| true| Dongan Church| 17|37.592888|127.056766|10110|37.574552|127.039721| 21| 31| 4| 1.06| 17.26| 6.7| 832| 117|
 Seoul|from other city|1000009| true|Coupang Logistics...| 25| -| -| null| null| null| null| null| null| null| null| null| null| 125|
 Seoul| Gwanak-gu|1000010| true| Wangsung Church| 30|37.481735|126.930121|10050| 37.47829|126.951502| 22| 33| 1| 0.89| 15.12| 4.9| 909| 130|
 Seoul| Eunpyeong-gu|1000011| true|Eunpyeong St. Mar...| 14| 37.63369| 126.9165|10220|37.603481|126.929173| 31| 44| 1| 1.09| 17.0| 6.5| 874| 114|
 Seoul| Seongdong-gu|1000012| true| Seongdong-gu APT| 13| 37.55713| 127.0403|10160|37.563277|127.036647| 21| 30| 2| 0.97| 14.76| 5.3| 593| 113|
 Seoul| Jongno-gu|1000013| true|Jongno Community ...| 10| 37.57681| 127.006|10230|37.572999|126.979189| 13| 17| 3| 1.71| 18.27| 6.8| 668| 110|
 Seoul| Gangnam-gu|1000014| true|Samsung Medical C...| 7| 37.48825| 127.08559|10010|37.518421|127.047222| 33| 38| 0| 4.18| 13.17| 4.3| 3088| 107|
 Seoul| Jung-gu|1000015| true|Jung-gu Fashion C...| 7|37.562405|126.984377|10240|37.563988| 126.99753| 12| 14| 2| 0.94| 18.42| 7.4| 728| 107|
 Seoul| Seodaemun-gu|1000016| true| Yeonana News Class| 5|37.558147|126.943799|10140|37.579428|126.936771| 19| 25| 6| 1.12| 16.77| 6.2| 587| 105|
 Seoul| Jongno-gu|1000017| true|Korea Campus Crus...| 7|37.594782|126.968022|10230|37.572999|126.979189| 13| 17| 3| 1.71| 18.27| 6.8| 668| 107|
 Seoul| Gangnam-gu|1000018| true|Gangnam Yeoksam-d...| 6| -| -|10010|37.518421|127.047222| 33| 38| 0| 4.18| 13.17| 4.3| 3088| 106|
 Seoul|from other city|1000019| true|Daejeon door-to-d...| 1| -| -| null| null| null| null| null| null| null| null| null| null| 101|
 Seoul| Geumcheon-gu|1000020| true|Geumcheon-gu rice...| 6| -| -|10080|37.456852|126.895229| 18| 19| 0| 0.96| 16.15| 6.7| 475| 106|
+--------+---------------+-------+-----+--------------------+---------+---------+----------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+------------+
only showing top 20 rows

We can also use math functions like F.exp function:

In [0]:
casesWithExpConfirmed = cases.withColumn("ExpConfirmed", F.exp("confirmed"))
casesWithExpConfirmed.show()

+--------+---------------+-------+-----+--------------------+---------+---------+----------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+--------------------+
province| city|case_id|group| infection_case|confirmed| latitude| longitude| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count| ExpConfirmed|
+--------+---------------+-------+-----+--------------------+---------+---------+----------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+--------------------+
 Seoul| Yongsan-gu|1000001| true| Itaewon Clubs| 139|37.538621|126.992652|10210|37.532768|126.990021| 15| 13| 1| 0.68| 16.87| 6.5| 435|2.327732040478862E60|
 Seoul| Gwanak-gu|1000002| true| Richway| 119| 37.48208|126.901384|10050| 37.47829|126.951502| 22| 33| 1| 0.89| 15.12| 4.9| 909|4.797813327299302E51|
 Seoul| Guro-gu|1000003| true| Guro-gu Call Center| 95|37.508163|126.884387|10070|37.495632| 126.88765| 26| 34| 3| 1.0| 16.21| 5.7| 741|1.811239082889023...|
 Seoul| Yangcheon-gu|1000004| true|Yangcheon Table T...| 43|37.546061|126.874209|10190|37.517189|126.866618| 30| 43| 0| 2.26| 13.55| 5.5| 816|4.727839468229346...|
 Seoul| Dobong-gu|1000005| true| Day Care Center| 43|37.679422|127.044374|10100|37.668952|127.047082| 23| 26| 1| 0.95| 17.89| 7.2| 485|4.727839468229346...|
 Seoul| Guro-gu|1000006| true|Manmin Central Ch...| 41|37.481059|126.894343|10070|37.495632| 126.88765| 26| 34| 3| 1.0| 16.21| 5.7| 741|6.398434935300549...|
 Seoul|from other city|1000007| true|SMR Newly Planted...| 36| -| -| null| null| null| null| null| null| null| null| null| null|4.311231547115195E15|
 Seoul| Dongdaemun-gu|1000008| true| Dongan Church| 17|37.592888|127.056766|10110|37.574552|127.039721| 21| 31| 4| 1.06| 17.26| 6.7| 832| 2.41549527535753E7|
 Seoul|from other city|1000009| true|Coupang Logistics...| 25| -| -| null| null| null| null| null| null| null| null| null| null|7.200489933738588E10|
 Seoul| Gwanak-gu|1000010| true| Wangsung Church| 30|37.481735|126.930121|10050| 37.47829|126.951502| 22| 33| 1| 0.89| 15.12| 4.9| 909|1.068647458152446...|
 Seoul| Eunpyeong-gu|1000011| true|Eunpyeong St. Mar...| 14| 37.63369| 126.9165|10220|37.603481|126.929173| 31| 44| 1| 1.09| 17.0| 6.5| 874| 1202604.2841647768|
 Seoul| Seongdong-gu|1000012| true| Seongdong-gu APT| 13| 37.55713| 127.0403|10160|37.563277|127.036647| 21| 30| 2| 0.97| 14.76| 5.3| 593| 442413.3920089205|
 Seoul| Jongno-gu|1000013| true|Jongno Community ...| 10| 37.57681| 127.006|10230|37.572999|126.979189| 13| 17| 3| 1.71| 18.27| 6.8| 668| 22026.465794806718|
 Seoul| Gangnam-gu|1000014| true|Samsung Medical C...| 7| 37.48825| 127.08559|10010|37.518421|127.047222| 33| 38| 0| 4.18| 13.17| 4.3| 3088| 1096.6331584284585|
 Seoul| Jung-gu|1000015| true|Jung-gu Fashion C...| 7|37.562405|126.984377|10240|37.563988| 126.99753| 12| 14| 2| 0.94| 18.42| 7.4| 728| 1096.6331584284585|
 Seoul| Seodaemun-gu|1000016| true| Yeonana News Class| 5|37.558147|126.943799|10140|37.579428|126.936771| 19| 25| 6| 1.12| 16.77| 6.2| 587| 148.4131591025766|
 Seoul| Jongno-gu|1000017| true|Korea Campus Crus...| 7|37.594782|126.968022|10230|37.572999|126.979189| 13| 17| 3| 1.71| 18.27| 6.8| 668| 1096.6331584284585|
 Seoul| Gangnam-gu|1000018| true|Gangnam Yeoksam-d...| 6| -| -|10010|37.518421|127.047222| 33| 38| 0| 4.18| 13.17| 4.3| 3088| 403.4287934927351|
 Seoul|from other city|1000019| true|Daejeon door-to-d...| 1| -| -| null| null| null| null| null| null| null| null| null| null| 2.7182818284590455|
 Seoul| Geumcheon-gu|1000020| true|Geumcheon-gu rice...| 6| -| -|10080|37.456852|126.895229| 18| 19| 0| 0.96| 16.15| 6.7| 475| 403.4287934927351|
+--------+---------------+-------+-----+--------------------+---------+---

#### [2] Using Spark UDFs

Sometimes we want to do complicated things to a column or multiple columns. This could be thought of as a map operation on a PySpark Dataframe to a single column or multiple columns. While Spark SQL functions do solve many use cases when it comes to column creation, I use Spark UDF whenever I need more matured Python functionality. \

To use Spark UDFs, we need to use the F.udf function to convert a regular python function to a Spark UDF. We also need to specify the return type of the function. In this example the return type is StringType()

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

def casesHighLow(confirmed):
    if confirmed < 50: 
        return 'low'
    else:
        return 'high'
    
#convert to a UDF Function by passing in the function and return type of function
casesHighLowUDF = F.udf(casesHighLow, StringType())
CasesWithHighLow = cases.withColumn("HighLow", casesHighLowUDF("confirmed"))
CasesWithHighLow.show()

+--------+---------------+-------+-----+--------------------+---------+---------+----------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-------+
province| city|case_id|group| infection_case|confirmed| latitude| longitude| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|HighLow|
+--------+---------------+-------+-----+--------------------+---------+---------+----------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-------+
 Seoul| Yongsan-gu|1000001| true| Itaewon Clubs| 139|37.538621|126.992652|10210|37.532768|126.990021| 15| 13| 1| 0.68| 16.87| 6.5| 435| high|
 Seoul| Gwanak-gu|1000002| true| Richway| 119| 37.48208|126.901384|10050| 37.47829|126.951502| 22| 33| 1| 0.89| 15.12| 4.9| 909| high|
 Seoul| Guro-gu|1000003| true| Guro-gu Call Center| 95|37.508163|126.884387|10070|37.495632| 126.88765| 26| 34| 3| 1.0| 16.21| 5.7| 741| high|
 Seoul| Yangcheon-gu|1000004| true|Yangcheon Table T...| 43|37.546061|126.874209|10190|37.517189|126.866618| 30| 43| 0| 2.26| 13.55| 5.5| 816| low|
 Seoul| Dobong-gu|1000005| true| Day Care Center| 43|37.679422|127.044374|10100|37.668952|127.047082| 23| 26| 1| 0.95| 17.89| 7.2| 485| low|
 Seoul| Guro-gu|1000006| true|Manmin Central Ch...| 41|37.481059|126.894343|10070|37.495632| 126.88765| 26| 34| 3| 1.0| 16.21| 5.7| 741| low|
 Seoul|from other city|1000007| true|SMR Newly Planted...| 36| -| -| null| null| null| null| null| null| null| null| null| null| low|
 Seoul| Dongdaemun-gu|1000008| true| Dongan Church| 17|37.592888|127.056766|10110|37.574552|127.039721| 21| 31| 4| 1.06| 17.26| 6.7| 832| low|
 Seoul|from other city|1000009| true|Coupang Logistics...| 25| -| -| null| null| null| null| null| null| null| null| null| null| low|
 Seoul| Gwanak-gu|1000010| true| Wangsung Church| 30|37.481735|126.930121|10050| 37.47829|126.951502| 22| 33| 1| 0.89| 15.12| 4.9| 909| low|
 Seoul| Eunpyeong-gu|1000011| true|Eunpyeong St. Mar...| 14| 37.63369| 126.9165|10220|37.603481|126.929173| 31| 44| 1| 1.09| 17.0| 6.5| 874| low|
 Seoul| Seongdong-gu|1000012| true| Seongdong-gu APT| 13| 37.55713| 127.0403|10160|37.563277|127.036647| 21| 30| 2| 0.97| 14.76| 5.3| 593| low|
 Seoul| Jongno-gu|1000013| true|Jongno Community ...| 10| 37.57681| 127.006|10230|37.572999|126.979189| 13| 17| 3| 1.71| 18.27| 6.8| 668| low|
 Seoul| Gangnam-gu|1000014| true|Samsung Medical C...| 7| 37.48825| 127.08559|10010|37.518421|127.047222| 33| 38| 0| 4.18| 13.17| 4.3| 3088| low|
 Seoul| Jung-gu|1000015| true|Jung-gu Fashion C...| 7|37.562405|126.984377|10240|37.563988| 126.99753| 12| 14| 2| 0.94| 18.42| 7.4| 728| low|
 Seoul| Seodaemun-gu|1000016| true| Yeonana News Class| 5|37.558147|126.943799|10140|37.579428|126.936771| 19| 25| 6| 1.12| 16.77| 6.2| 587| low|
 Seoul| Jongno-gu|1000017| true|Korea Campus Crus...| 7|37.594782|126.968022|10230|37.572999|126.979189| 13| 17| 3| 1.71| 18.27| 6.8| 668| low|
 Seoul| Gangnam-gu|1000018| true|Gangnam Yeoksam-d...| 6| -| -|10010|37.518421|127.047222| 33| 38| 0| 4.18| 13.17| 4.3| 3088| low|
 Seoul|from other city|1000019| true|Daejeon door-to-d...| 1| -| -| null| null| null| null| null| null| null| null| null| null| low|
 Seoul| Geumcheon-gu|1000020| true|Geumcheon-gu rice...| 6| -| -|10080|37.456852|126.895229| 18| 19| 0| 0.96| 16.15| 6.7| 475| low|
+--------+---------------+-------+-----+--------------------+---------+---------+----------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-------+
only showing top 20 rows

#### [3] Using Pandas UDF

This allows you to use pandas functionality with Spark. I generally use it when I have to run a groupBy operation on a Spark dataframe or whenever I need to create rolling features
 
The way we use it is by using the F.pandas_udf decorator. **We assume here that the input to the function will be a pandas data frame**

The only complexity here is that we have to provide a schema for the output Dataframe. We can use the original schema of a dataframe to create the outSchema.

In [0]:
cases.printSchema()

root
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- confirmed: integer (nullable = true)
 |-- code: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elementary_school_count: integer (nullable = true)
 |-- kindergarten_count: integer (nullable = true)
 |-- university_count: integer (nullable = true)
 |-- academy_ratio: double (nullable = true)
 |-- elderly_population_ratio: double (nullable = true)
 |-- elderly_alone_ratio: double (nullable = true)
 |-- nursing_home_count: integer (nullable = true)



In [0]:
from pyspark.sql.types import IntegerType, StringType, DoubleType, BooleanType
from pyspark.sql.types import StructType, StructField

# Declare the schema for the output of our function

outSchema = StructType([StructField('case_id',IntegerType(),True),
                        StructField('province',StringType(),True),
                        StructField('city',StringType(),True),
                        StructField('group',BooleanType(),True),
                        StructField('infection_case',StringType(),True),
                        StructField('confirmed',IntegerType(),True),
                        StructField('latitude',StringType(),True),
                        StructField('longitude',StringType(),True),
                        StructField('normalized_confirmed',DoubleType(),True)
                       ])
# decorate our function with pandas_udf decorator
@F.pandas_udf(outSchema, F.PandasUDFType.GROUPED_MAP)
def subtract_mean(pdf):
    # pdf is a pandas.DataFrame
    v = pdf.confirmed
    v = v - v.mean()
    pdf['normalized_confirmed'] = v
    return pdf

confirmed_groupwise_normalization = cases.groupby("infection_case").apply(subtract_mean)

confirmed_groupwise_normalization.limit(10).toPandas()

/databricks/spark/python/pyspark/sql/pandas/group_ops.py:81: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
 warnings.warn(

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-751677916595998> in <module> 
 23 return pdf
 24 
 ---> 25 confirmed_groupwise_normalization = cases . groupby ( "infection_case" ) . apply ( subtract_mean ) 
 26 
 27 confirmed_groupwise_normalization . limit ( 10 ) . toPandas ( ) 

 /databricks/spark/python/pyspark/sql/pandas/group_ops.py in apply (self, udf) 
 84 "more details.", UserWarning)
 85 
 ---> 86 return self . applyInPandas ( udf . func , schema = udf . returnType ) 
 87 
 88 def applyInPandas ( self , func , schema ) : 

 /databricks/spark/python/pyspark/sql/pandas/group_ops.py in applyInPandas (self, func, schema) 
 199 func, returnType=schema, functionType=PandasUDFType.GROUPED_MAP)
 200 df = self . _df
 --> 201 udf_column = udf ( * [ df [ col ] for col in df . columns ] ) 
 202 jdf = self . _jgd . flatMapGroupsInPandas ( udf_column . _jc . expr ( ) ) 
 203 return DataFrame ( jdf , self . sql_ctx ) 

 /databricks/spark/python/pyspark/sql/pandas/group_ops.py in <listcomp> (.0) 
 199 func, returnType=schema, functionType=PandasUDFType.GROUPED_MAP)
 200 df = self . _df
 --> 201 udf_column = udf ( * [ df [ col ] for col in df . columns ] ) 
 202 jdf = self . _jgd . flatMapGroupsInPandas ( udf_column . _jc . expr ( ) ) 
 203 return DataFrame ( jdf , self . sql_ctx ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in __getitem__ (self, item) 
 1641 """
 1642 if isinstance ( item , str ) : 
 -> 1643 jc = self . _jdf . apply ( item ) 
 1644 return Column ( jc ) 
 1645 elif isinstance ( item , Column ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Reference 'latitude' is ambiguous, could be: latitude, latitude.

### 4. Spark Window Functions
We will simply look at some of the most important and useful window functions available.

In [0]:
timeprovince = spark.read.load("dbfs:/FileStore/Module4/TimeProvince.csv",
                          format="csv", 
                          sep=",", 
                          inferSchema="true", 
                          header="true")

timeprovince.show()

+----------+----+-----------------+---------+--------+--------+
 date|time| province|confirmed|released|deceased|
+----------+----+-----------------+---------+--------+--------+
2020-01-20| 16| Seoul| 0| 0| 0|
2020-01-20| 16| Busan| 0| 0| 0|
2020-01-20| 16| Daegu| 0| 0| 0|
2020-01-20| 16| Incheon| 1| 0| 0|
2020-01-20| 16| Gwangju| 0| 0| 0|
2020-01-20| 16| Daejeon| 0| 0| 0|
2020-01-20| 16| Ulsan| 0| 0| 0|
2020-01-20| 16| Sejong| 0| 0| 0|
2020-01-20| 16| Gyeonggi-do| 0| 0| 0|
2020-01-20| 16| Gangwon-do| 0| 0| 0|
2020-01-20| 16|Chungcheongbuk-do| 0| 0| 0|
2020-01-20| 16|Chungcheongnam-do| 0| 0| 0|
2020-01-20| 16| Jeollabuk-do| 0| 0| 0|
2020-01-20| 16| Jeollanam-do| 0| 0| 0|
2020-01-20| 16| Gyeongsangbuk-do| 0| 0| 0|
2020-01-20| 16| Gyeongsangnam-do| 0| 0| 0|
2020-01-20| 16| Jeju-do| 0| 0| 0|
2020-01-21| 16| Seoul| 0| 0| 0|
2020-01-21| 16| Busan| 0| 0| 0|
2020-01-21| 16| Daegu| 0| 0| 0|
+----------+----+-----------------+---------+--------+--------+
only showing top 20 rows

#### [1] Ranking

You can get rank as well as dense_rank on a group using this function. For example, you may want to have a column in your cases table that provides the rank of infection_case based on the number of infection_case in a province. We can do this by:

In [0]:
from pyspark.sql.window import Window
windowSpec = Window().partitionBy(['province']).orderBy(F.desc('confirmed'))
cases.withColumn("rank",F.rank().over(windowSpec)).show()

+-----------------+---------------+-------+-----+--------------------+---------+--------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+----+
 province| city|case_id|group| infection_case|confirmed|latitude|longitude| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|rank|
+-----------------+---------------+-------+-----+--------------------+---------+--------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+----+
 Busan| Dongnae-gu|1100001| true| Onchun Church| 39|35.21628| 129.0771|11060| 35.20506|129.083673| 22| 31| 0| 1.98| 17.53| 7.7| 608| 1|
 Busan| -|1100008|false| overseas inflow| 36| -| -| null| null| null| null| null| null| null| null| null| null| 2|
 Busan| -|1100010|false| etc| 30| -| -| null| null| null| null| null| null| null| null| null| null| 3|
 Busan| -|1100009|false|contact with patient| 19| -| -| null| null| null| null| null| null| null| null| null| null| 4|
 Busan|from other city|1100002| true| Shincheonji Church| 12| -| -| null| null| null| null| null| null| null| null| null| null| 5|
 Busan| Haeundae-gu|1100004| true|Haeundae-gu Catho...| 6|35.20599| 129.1256|11160| 35.16336|129.163594| 33| 39| 1| 1.63| 16.53| 7.9| 814| 6|
 Busan| Suyeong-gu|1100003| true|Suyeong-gu Kinder...| 5|35.16708| 129.1124|11120|35.145805|129.113194| 10| 22| 0| 1.56| 20.4| 8.2| 395| 7|
 Busan| Jin-gu|1100005| true| Jin-gu Academy| 4|35.17371| 129.0633| null| null| null| null| null| null| null| null| null| null| 8|
 Busan|from other city|1100006| true| Itaewon Clubs| 4| -| -| null| null| null| null| null| null| null| null| null| null| 8|
 Busan|from other city|1100007| true|Cheongdo Daenam H...| 1| -| -| null| null| null| null| null| null| null| null| null| null| 10|
Chungcheongbuk-do| -|4000005|false| overseas inflow| 13| -| -| null| null| null| null| null| null| null| null| null| null| 1|
Chungcheongbuk-do| Goesan-gun|4000001| true|Goesan-gun Jangye...| 11|36.82422| 127.9552|40010| 36.81534|127.786651| 14| 15| 1| 0.36| 33.01| 16.5| 64| 2|
Chungcheongbuk-do| -|4000007|false| etc| 11| -| -| null| null| null| null| null| null| null| null| null| null| 2|
Chungcheongbuk-do|from other city|4000002| true| Itaewon Clubs| 9| -| -| null| null| null| null| null| null| null| null| null| null| 4|
Chungcheongbuk-do| -|4000006|false|contact with patient| 8| -| -| null| null| null| null| null| null| null| null| null| null| 5|
Chungcheongbuk-do|from other city|4000004| true| Shincheonji Church| 6| -| -| null| null| null| null| null| null| null| null| null| null| 6|
Chungcheongbuk-do|from other city|4000003| true| Guro-gu Call Center| 2| -| -| null| null| null| null| null| null| null| null| null| null| 7|
Chungcheongnam-do| Cheonan-si|4100001| true|gym facility in C...| 103|36.81503| 127.1139|41120| 36.81498|127.113868| 75| 112| 6| 1.91| 10.42| 4.5| 1069| 1|
Chungcheongnam-do| -|4100006|false| overseas inflow| 16| -| -| null| null| null| null| null| null| null| null| null| null| 2|
Chungcheongnam-do| -|4100008|false| etc| 12| -| -| null| null| null| null| null| null| null| null| null| null| 3|
+-----------------+---------------+-------+-----+--------------------+---------+--------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+----+
only showing top 20 rows

#### [2] Lag Variables

Sometimes our data science models may need **lag based** features. For example, a model might have variables like the price last week or sales quantity the previous day. We can create such features using the lag function with window functions. \

Here I am trying to get the confirmed cases 7 days before. I am filtering to show the results as the first few days of corona cases were zeros. You can see here that the lag_7 day feature is shifted by 7 days.

In [0]:
from pyspark.sql.window import Window

windowSpec = Window().partitionBy(['province']).orderBy('date')

timeprovinceWithLag = timeprovince.withColumn("lag_7",F.lag("confirmed", 7).over(windowSpec))

timeprovinceWithLag.filter(timeprovinceWithLag.date>'2020-03-10').show()

+----------+----+--------+---------+--------+--------+-----+
 date|time|province|confirmed|released|deceased|lag_7|
+----------+----+--------+---------+--------+--------+-----+
2020-03-11| 0| Busan| 98| 21| 0| 92|
2020-03-12| 0| Busan| 99| 29| 0| 92|
2020-03-13| 0| Busan| 100| 36| 0| 95|
2020-03-14| 0| Busan| 103| 40| 0| 96|
2020-03-15| 0| Busan| 106| 52| 1| 96|
2020-03-16| 0| Busan| 107| 53| 1| 96|
2020-03-17| 0| Busan| 107| 54| 1| 96|
2020-03-18| 0| Busan| 107| 58| 1| 98|
2020-03-19| 0| Busan| 107| 58| 1| 99|
2020-03-20| 0| Busan| 108| 60| 1| 100|
2020-03-21| 0| Busan| 108| 67| 1| 103|
2020-03-22| 0| Busan| 108| 69| 1| 106|
2020-03-23| 0| Busan| 109| 71| 1| 107|
2020-03-24| 0| Busan| 111| 71| 1| 107|
2020-03-25| 0| Busan| 112| 73| 1| 107|
2020-03-26| 0| Busan| 112| 75| 2| 107|
2020-03-27| 0| Busan| 113| 81| 2| 108|
2020-03-28| 0| Busan| 114| 81| 2| 108|
2020-03-29| 0| Busan| 117| 85| 3| 108|
2020-03-30| 0| Busan| 118| 87| 3| 109|
+----------+----+--------+---------+--------+--------+-----+
only showing top 20 rows

#### [3] Rolling Aggregations

For example, we might want to have a rolling 7-day sales sum/mean as a feature for our sales regression model. Let us calculate the rolling mean of confirmed cases for the last 7 days here. This is what a lot of the people are already doing with this dataset to see the real trends.

In [0]:
from pyspark.sql.window import Window

# we only look at the past 7 days in a particular window including the current_day. 
# Here 0 specifies the current_row and -6 specifies the seventh row previous to current_row. 
# Remember we count starting from 0.

# If we had used rowsBetween(-7,-1), we would just have looked at past 7 days of data and not the current_day
windowSpec = Window().partitionBy(['province']).orderBy('date').rowsBetween(-6,0)

timeprovinceWithRoll = timeprovince.withColumn("roll_7_confirmed",F.mean("confirmed").over(windowSpec))

timeprovinceWithRoll.filter(timeprovinceWithLag.date>'2020-03-10').show()

+----------+----+--------+---------+--------+--------+------------------+
 date|time|province|confirmed|released|deceased| roll_7_confirmed|
+----------+----+--------+---------+--------+--------+------------------+
2020-03-11| 0| Busan| 98| 21| 0| 95.57142857142857|
2020-03-12| 0| Busan| 99| 29| 0| 96.57142857142857|
2020-03-13| 0| Busan| 100| 36| 0| 97.28571428571429|
2020-03-14| 0| Busan| 103| 40| 0| 98.28571428571429|
2020-03-15| 0| Busan| 106| 52| 1| 99.71428571428571|
2020-03-16| 0| Busan| 107| 53| 1|101.28571428571429|
2020-03-17| 0| Busan| 107| 54| 1|102.85714285714286|
2020-03-18| 0| Busan| 107| 58| 1|104.14285714285714|
2020-03-19| 0| Busan| 107| 58| 1|105.28571428571429|
2020-03-20| 0| Busan| 108| 60| 1|106.42857142857143|
2020-03-21| 0| Busan| 108| 67| 1|107.14285714285714|
2020-03-22| 0| Busan| 108| 69| 1|107.42857142857143|
2020-03-23| 0| Busan| 109| 71| 1|107.71428571428571|
2020-03-24| 0| Busan| 111| 71| 1|108.28571428571429|
2020-03-25| 0| Busan| 112| 73| 1| 109.0|
2020-03-26| 0| Busan| 112| 75| 2|109.71428571428571|
2020-03-27| 0| Busan| 113| 81| 2|110.42857142857143|
2020-03-28| 0| Busan| 114| 81| 2|111.28571428571429|
2020-03-29| 0| Busan| 117| 85| 3|112.57142857142857|
2020-03-30| 0| Busan| 118| 87| 3|113.85714285714286|
+----------+----+--------+---------+--------+--------+------------------+
only showing top 20 rows

One could also find a use for **rowsBetween(Window.unboundedPreceding, Window.currentRow)** function, where we take the rows between the first row in a window and the current_row to get running totals. I am calculating cumulative_confirmed here.

In [0]:
from pyspark.sql.window import Window

windowSpec = Window().partitionBy(['province']).orderBy('date').rowsBetween(Window.unboundedPreceding,Window.currentRow)

timeprovinceWithRoll = timeprovince.withColumn("cumulative_confirmed",F.sum("confirmed").over(windowSpec))

timeprovinceWithRoll.filter(timeprovinceWithLag.date>'2020-03-10').show()

+----------+----+--------+---------+--------+--------+--------------------+
 date|time|province|confirmed|released|deceased|cumulative_confirmed|
+----------+----+--------+---------+--------+--------+--------------------+
2020-03-11| 0| Busan| 98| 21| 0| 1408|
2020-03-12| 0| Busan| 99| 29| 0| 1507|
2020-03-13| 0| Busan| 100| 36| 0| 1607|
2020-03-14| 0| Busan| 103| 40| 0| 1710|
2020-03-15| 0| Busan| 106| 52| 1| 1816|
2020-03-16| 0| Busan| 107| 53| 1| 1923|
2020-03-17| 0| Busan| 107| 54| 1| 2030|
2020-03-18| 0| Busan| 107| 58| 1| 2137|
2020-03-19| 0| Busan| 107| 58| 1| 2244|
2020-03-20| 0| Busan| 108| 60| 1| 2352|
2020-03-21| 0| Busan| 108| 67| 1| 2460|
2020-03-22| 0| Busan| 108| 69| 1| 2568|
2020-03-23| 0| Busan| 109| 71| 1| 2677|
2020-03-24| 0| Busan| 111| 71| 1| 2788|
2020-03-25| 0| Busan| 112| 73| 1| 2900|
2020-03-26| 0| Busan| 112| 75| 2| 3012|
2020-03-27| 0| Busan| 113| 81| 2| 3125|
2020-03-28| 0| Busan| 114| 81| 2| 3239|
2020-03-29| 0| Busan| 117| 85| 3| 3356|
2020-03-30| 0| Busan| 118| 87| 3| 3474|
+----------+----+--------+---------+--------+--------+--------------------+
only showing top 20 rows

### 5. Pivot DataFrames

Sometimes we may need to have the dataframe in flat format. This happens frequently in movie data where we may want to show genres as columns instead of rows. We can use pivot to do this. Here I am trying to get one row for each date and getting the province names as columns.

In [0]:
pivotedTimeprovince = timeprovince.groupBy('date').pivot('province') \
.agg(F.sum('confirmed').alias('confirmed') , F.sum('released').alias('released'))

pivotedTimeprovince.limit(10).toPandas()

Out[28]:

,date,Busan_confirmed,Busan_released,Chungcheongbuk-do_confirmed,Chungcheongbuk-do_released,Chungcheongnam-do_confirmed,Chungcheongnam-do_released,Daegu_confirmed,Daegu_released,Daejeon_confirmed,Daejeon_released,Gangwon-do_confirmed,Gangwon-do_released,Gwangju_confirmed,Gwangju_released,Gyeonggi-do_confirmed,Gyeonggi-do_released,Gyeongsangbuk-do_confirmed,Gyeongsangbuk-do_released,Gyeongsangnam-do_confirmed,Gyeongsangnam-do_released,Incheon_confirmed,Incheon_released,Jeju-do_confirmed,Jeju-do_released,Jeollabuk-do_confirmed,Jeollabuk-do_released,Jeollanam-do_confirmed,Jeollanam-do_released,Sejong_confirmed,Sejong_released,Seoul_confirmed,Seoul_released,Ulsan_confirmed,Ulsan_released
0,2020-04-13,126,103,45,31,139,109,6819,5395,39,23,49,28,27,19,631,305,1337,1020,115,84,87,39,12,5,17,8,15,6,46,22,610,214,41,33
1,2020-02-26,58,0,5,0,3,0,710,1,5,0,6,0,9,2,51,8,317,1,34,0,3,1,2,0,3,1,1,0,1,0,49,8,4,0
2,2020-06-24,152,142,62,56,162,146,6903,6687,94,44,63,52,33,32,1137,761,1386,1321,133,124,333,174,19,15,25,20,20,18,49,47,1241,747,55,48
3,2020-06-08,147,141,61,49,148,143,6888,6641,46,43,58,51,32,30,942,680,1383,1309,124,122,283,118,15,13,21,19,20,17,47,47,996,651,53,46
4,2020-06-22,150,142,61,56,161,144,6900,6681,82,44,62,52,33,32,1123,753,1385,1320,133,124,329,169,19,15,24,20,20,18,49,47,1224,733,53,48
5,2020-06-20,150,142,61,55,158,144,6898,6679,72,44,60,52,32,32,1107,741,1384,1319,132,124,328,165,19,15,23,20,20,18,48,47,1202,730,53,48
6,2020-04-12,126,103,45,29,139,108,6816,5356,39,23,49,27,27,19,628,292,1333,1013,115,84,86,38,12,4,17,8,15,5,46,22,602,202,41,32
7,2020-02-13,0,0,0,0,0,0,0,0,0,0,0,0,1,0,12,5,0,1,0,0,1,1,0,0,1,1,0,0,0,0,14,2,0,0
8,2020-04-20,132,115,45,38,141,122,6833,5769,39,26,53,28,30,22,656,387,1361,1064,116,87,92,56,13,6,17,9,15,8,46,29,624,304,43,34
9,2020-06-09,147,141,61,49,150,143,6888,6643,46,43,58,51,32,30,955,680,1383,1310,125,123,286,121,15,13,21,19,20,17,47,47,1015,664,53,46


### 6. Other Opertions

#### [1] Caching

Spark works on the lazy execution principle. What that means is that nothing really gets executed until you use an action function like the .count() on a dataframe. And if you do a .count function, it generally helps to cache at this step. So I have made it a point to cache() my dataframes whenever I do a .count() operation.

In [0]:

timeprovinceWithRoll.cache().count()

Out[30]: 2771

#### [2] Save and Load from an intermediate step

When you work with Spark you will frequently run with memory and storage issues. While in some cases such issues might be resolved using techniques like broadcasting, salting or cache, sometimes just interrupting the workflow and saving and reloading the whole dataframe at a crucial step has helped me a lot. This helps spark to let go of a lot of memory that gets utilized for storing intermediate shuffle data and unused caches.

In [0]:
timeprovinceWithRoll.write.mode("overwrite").parquet("dbfs:/FileStore/Module4/outputdata/df.parquet")
timeprovinceWithRoll.unpersist()
# spark.read.load("dbfs:/FileStore/Module4/outputdata/df.parquet")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-751677916596022> in <module> 
 ----> 1 timeprovinceWithRoll . write . mode ( "overwrite" ) . parquet ( "dbfs:/FileStore/Module4/outputdata/df.parquet" ) 
 2 timeprovinceWithRoll . unpersist ( ) 
 3 # spark.read.load("dbfs:/FileStore/Module4/outputdata/df.parquet") 

 NameError : name 'timeprovinceWithRoll' is not defined

#### [3] Repartitioning

In [0]:
# You might want to repartition your data if you feel your data has been skewed while working with all the transformations and joins. 
# The simplest way to do it is by using:
timeprovinceWithRoll = timeprovinceWithRoll.repartition(1000)

In [0]:
#Sometimes you might also want to repartition by a known scheme as this scheme might be used by a certain join or aggregation operation later on. # You can use multiple columns to repartition using:
timeprovinceWithRoll = timeprovinceWithRoll.repartition('date', 'time')

In [0]:
# Then, we can get the number of partitions in a data frame using:
timeprovinceWithRoll.rdd.getNumPartitions()

## LOL :) 1 :D

Out[37]: 1

#### [4] Reading Parquet File in Local
Sometimes you might want to read the parquet files in a system where Spark is not available. In such cases, I normally use the below code:

In [0]:
from glob import glob
def load_df_from_parquet(parquet_directory):
    df = pd.DataFrame()
    for file in glob(f"{parquet_directory}/*"):
        df = pd.concat([df,pd.read_parquet(file)])
    return df

In [0]:
load_df_from_parquet("dbfs:/FileStore/Module4/outputdata/df.parquet")

Out[41]:

""


### 7. Close Spark Instance

In [0]:
spark.stop()